In [1042]:
# import packages 
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import re
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import nltk
from wordcloud import WordCloud
from textblob import TextBlob
from nltk.corpus import wordnet
#nltk.download('punkt')
#nltk.download('averaged_perceptron_tagger')
#nltk.download('brown')

In [918]:
# set settings
%matplotlib inline
plt.style.use('ggplot')
pd.set_option('display.max_rows', 100)
2
# Width of the display in characters. If set to None and pandas will correctly auto-detect the width.
pd.set_option('display.max_colwidth', 150)

In [888]:
#df = pd.read_csv('https://s3.amazonaws.com/nycdsabt01/movie_metadata.csv')

In [889]:
rc = pd.read_csv("simplyrecipes.csv")

In [890]:
# missing values
# if dish is missing, discard data
rc.dropna(subset=['dish'], inplace=True)
# drop column with no ingredients
rc.dropna(subset=['ingredients'], inplace=True)
# if cook time missing, discard data
rc = rc[rc[['time_cook', 'time_prep']].notna().any(axis = 1)]
# drop num_comment column, all NaN
rc = rc.drop(columns = ['num_comment'])

In [891]:
# split cell content string to list
lst1 = ['time_cook', 'time_prep', 'tags']
rc[lst1] = rc[lst1].apply(lambda x: x.str.split(','))

In [892]:
# change rating star to numbers
rc.rating_star.unique() # only 5 starts
rc['rating_star'] = rc.rating_star.str.replace('yyyyy', '5')

In [1065]:
# clean ingredients with regex
# lower case all letters
rc['inglst'] = rc.ingredients.str.lower()
# get rid of number & punctuation 
rc['inglst'] = rc.inglst.apply(lambda x: re.sub('\W', " ", x))
rc['inglst'] = rc.inglst.apply(lambda x: re.sub('\d', " ", x))

In [1066]:
# remove common seasoning
lst4 = ["olive oil", "sea salt", "black pepper"]
rc['inglst'] = rc.inglst.apply(lambda x: re.sub("olive oil", "", x))
rc['inglst'] = rc.inglst.apply(lambda x: re.sub("sea salt", "", x))
rc['inglst'] = rc.inglst.apply(lambda x: re.sub("black pepper", "", x))
#rc.inglist.apply(lambda x: re.sub(lambda y: y, lst4), "", x)
# def remove_season(lst,x):
#     for item in lst:
#         re.sub(item, "", x)
# rc.inglst.apply(remove_season, lst4)

In [1067]:
# tokenize ingredients
def safe_word_tokenize(x):
    try:
        return word_tokenize(x)
    except:
        return 'empty'
#rc['inglst'] = rc.ingredients.apply(safe_word_tokenize)
rc['inglst'] = rc.inglst.apply(word_tokenize)

In [1068]:
# customize stop words list
lst2 = ['pinch','approx','inch','inches','cups','cup','ml','tablespoons','tablespoon','tbsp', 'teaspoon','kg','g','grams','gram','lbs','lb','pound','pounds','ounce','ounces','oz']
lst3 = ['salt', 'oil']
stopIng = stopwords.words('english')
stopIng.extend(lst2)
stopIng.extend(lst3)
#print(stopIng)

In [1069]:
# exlude stop words from inglst
rc['inglst'] = rc.inglst.apply(lambda x: [el for el in x if el not in stopIng])

In [1048]:
# blob = TextBlob(" ".join(rc.inglst.iloc[1]))
# print(blob.noun_phrases)

['extra virgin olive bunch lacinato kale dinosaur tuscan kale center ribs', 'black pepper cremini mushrooms', 'small yellow onion', 'loaf ciabatta bread', 'thick slices', 'large eggs ml milk', 'cheddar cheese']


In [1070]:
# only keep nouns in inglst
rc['inglst'] = rc.inglst.apply(nltk.pos_tag)
def only_noun(lst):
    nouns = [word for (word, pos) in lst if pos[:2]=="NN"]
    return nouns
rc['inglst'] = rc.inglst.apply(only_noun)

In [1075]:
#rc.ingredients.sample(5)
# only take the last two words of each , separated words in ingredients
#rc.sample(5)

In [ ]:
# parse time

In [ ]:
# parse tag

In [1018]:
# food = wordnet.synset('food.n.02')
# for i in food.closure(lambda s:s.hyponyms()):
#     print(i)

In [1073]:
# lines = 'lines is some string of words'
# tokenized = nltk.word_tokenize(lines)
# nouns = [word for (word, pos) in nltk.pos_tag(tokenized) if(pos[:2] == 'NN')]
# nouns = [(word, pos) for (word, pos) in nltk.pos_tag(tokenized)]
# nltk.pos_tag(tokenized)

In [813]:
#rc.inglst.apply( lambda x: [el for el in x if (el not in stopIng)] )

0      [cauliflower, florets, teaspoon, sea, salt, olive, oil, divided, g, g, parmesan, cheese, omit, vegan, version, fresh...
2      [extra, virgin, olive, oil, bunch, g, lacinato, kale, dinosaur, tuscan, kale, center, ribs, removed, leaves, thinly,...
3      [butter, baking, dish, water, polenta, butter, cut, little, cubes, teaspoon, salt, cheese, italian, blend, oil, gril...
5      [cauliflower, chopped, bite, size, pieces, teaspoons, neutral, oil, sea, salt, inch, corn, tortillas, warmed, teaspo...
6      [extra, virgin, olive, oil, skinless, chicken, breast, cut, inch, cubes, salt, yellow, onion, roughly, chopped, heap...
                                                                ...                                                           
930    [boneless, skinless, chicken, breast, halves, total, chicken, cutlets, flour, teaspoon, salt, ground, black, pepper,...
931    [beef, shoulder, boneless, chuck, roast, look, piece, well, marbled, fat, best, results, extra, virgin, 

In [551]:
# test why wordtokenize did not work on column ingredients
# def safe_word_tokenize( x ):
#     try:
#         return word_tokenize( x )
#     except Exception as e:
#         print( f"Error with {x}. Exception given: {e}" )
#         print( "-"*25 + "\n" )
# rc.ingredients.apply(safe_word_tokenize)

In [464]:
#rc.sample(4)
#rc.num_comment.unique()
#rc.loc[861,'time_cook']

In [351]:
#recipes.head()
print(rc.shape)
rc.sample(3)[['time_cook', 'time_prep', 'time_other_full']]
#rc.columns

(935, 13)


,time_cook,time_prep,time_other_full
464,NaN,NaN,NaN
514,NaN,NaN,NaN
550,NaN,NaN,NaN


In [352]:
# extract time
#rc['time_cook'].apply(lambda x: x.split(','))
# rc['time_cook'] = rc['time_cook'].str.split(',')
# rc['time_prep'] = rc['time_prep'].str.split(',')
# rc['ingredients'] = rc.ingredients.str.split(',')
# rc['tags'] = rc.tags.str.split(',')
#rc.rating_star.str.replace('yyyyy', 5)
#rc.rating_star.str.replace('yyyyy', '5')

In [530]:
# tests
#pd.isnull(rc['dish'])
#pd.isnull(rc['dish']).sum()

# a = rc.loc[925]['time_prep']
# b = rc.loc[82]['time_cook']
# a.split(',')
# rc['time_other_full'].sample(10)

#rc['time_cook'].sample(10)
#print(pd.isnull(rc[['time_cook', 'time_prep']]).sum())
#pd.isnull(rc['time_cook'] and rc['time_prep']).any()
rc[['time_cook', 'time_prep']].isna().any(axis = 1).sum()
#rc.shape
rc[rc[['time_cook', 'time_prep']].notna().any(axis = 1)].shape


(814, 14)

In [57]:
# old wrong codes
#rc['inglst'].apply(lambda x: x if x not in stopIng else '')

Int64Index([  0,   2,   3,   4,   5,   6,   7,   8,   9,  10,
            ...
            925, 926, 927, 928, 929, 930, 931, 932, 933, 934],
           dtype='int64', length=857)